### Search Engine with Tools and Agents

arxiv==2.2.0
wikipedia==1.4.0

In [3]:
# Tools Creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [ ]:
# API Wrapper - Wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

In [6]:
# API Wrapper - Wikipedia
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

In [7]:
# Combine two tool
tools = [wiki, arxiv]

In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

In [10]:
# Create Custom Tool - RAG Tool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Users\LENOVO\OneDrive\Desktop\Project\GenAI\LangChain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents, embeddings)
retriever = vectordb.as_retriever()

In [12]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever, 'LangSmith-Sarch', 'Sarch any information about LangSmith')

In [13]:
tools = [wiki, arxiv, retriever_tool]

In [14]:
# Run all this tools with Agents and LLM Models
# Tools, LLM --> AgentExecutor
from langchain_groq import ChatGroq

groq_api_key = os.getenv('GROQ_API_KEY')

# Large Language Model
llm = ChatGroq(groq_api_key=groq_api_key, model_name='llama3-8b-8192')

In [15]:
# Prompt Template
from langchain import hub

prompt = hub.pull('hwchase17/openai-functions-agent')
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [16]:
# Agents
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)

In [18]:
# Agent Executer
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [19]:
agent_executor.invoke({'input': 'Tell me about LangSmith'})



> Entering new AgentExecutor chain...

Invoking: `LangSmith-Sarch` with `{'query': 'LangSmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from proto

{'input': 'Tell me about LangSmith',
 'output': "It looks like LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It provides observability features to help monitor and evaluate the application's performance, as well as tools for prompt engineering and evaluation.\n\nSome of the key features of LangSmith include:\n\n1. Observability: LangSmith provides LLM-native observability, allowing developers to get meaningful insights from their application.\n2. Evaluation: LangSmith allows developers to evaluate their application over production traffic, scoring performance and getting human feedback on the data.\n3. Prompt Engineering: LangSmith provides tools for iterating on prompts, with automatic version control and collaboration features.\n4. SDK and UI: LangSmith offers a software development kit (SDK) and user interface (UI) for building and running high-quality evaluations.\n\nOverall, LangSmith seems to be a powerful platform for building and

In [21]:
agent_executor.invoke({'input': 'What is machine learning?'})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witBased on the provided information, it seems that the user is interested in learning more about machine learning. Since we already used the Wikipedia tool to gather information, we can provide a direct response to the user's question.

Machine learning is a subfield of artificial intelligence that involves the use of algorithms to analyze and learn from data, allowing the algorithms to make predictions or decisions without being explicitly programmed. It is a key technology behind many modern applications, such as self-driving cars, personal assistants, and medical diagnosis systems.

With the help of machine learning, computers can learn pa

{'input': 'What is machine learning?',
 'output': "Based on the provided information, it seems that the user is interested in learning more about machine learning. Since we already used the Wikipedia tool to gather information, we can provide a direct response to the user's question.\n\nMachine learning is a subfield of artificial intelligence that involves the use of algorithms to analyze and learn from data, allowing the algorithms to make predictions or decisions without being explicitly programmed. It is a key technology behind many modern applications, such as self-driving cars, personal assistants, and medical diagnosis systems.\n\nWith the help of machine learning, computers can learn patterns and relationships in data, and use this knowledge to make predictions or decisions. For example, a machine learning algorithm can be trained on a dataset of images of cats and dogs, and then use this training to predict the type of animal in a new image.\n\nMachine learning has many applic

In [22]:
agent_executor.invoke({'input': 'What is the paper 1706.03762 about?'})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,The paper "Attention Is All You Need" presents the Transformer model, a new architecture for sequence-to-sequence tasks. It's a significant contribution to the field of natural language processing, and it has had a profound impact on many applications.

> Finished chain.


{'input': 'What is the paper 1706.03762 about?',
 'output': 'The paper "Attention Is All You Need" presents the Transformer model, a new architecture for sequence-to-sequence tasks. It\'s a significant contribution to the field of natural language processing, and it has had a profound impact on many applications.'}